# LightGBM
https://blog.csdn.net/owenfy/article/details/79631144
- learning_rate =0.1,
-       n_estimators= 109,  #第二轮参数调整得到的 n_estimators 最优值
-      max_depth=7,
-     min_child_weight=1,
-   gamma=0,
-  subsample=0.5,
-  colsample_bytree=0.6,
-  colsample_bylevel=0.7,
-  objective= 'multi:softprob',
-  n_jobs=4, 
-  seed=3)

In [1]:
import lightgbm as lgbm
import pandas as pd 
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [84]:
# 是否对 y_train 进行标准化 
Y_Standard = True
out_id = 1 #输出文件编号 

#读取训练数据: 
path  = '../data/new/'
path_train = '../data/new/train=0.1/'
path_test = '../data/new/test/'
summit_path = '../data/summit/'

train = pd.read_csv(path_train + 'train=0.1-fe-coor0.3-pay.csv')
train.head()

,user_id,sr_infantry_atk_level,magic_add_value,cavalry_add_value,sr_rss_e_gather_level,sr_cavalry_atk_level,sr_march_size_level,sr_outpost_tier_4_level,sr_guest_troop_capacity_level,sr_shaman_atk_level,...,wood_reduce_value,general_acceleration_add_value,wood_add_value,stone_add_value,general_acceleration_reduce_value,ivory_reduce_value,ivory_add_value,stone_reduce_value,pay_price,prediction_pay_price
0,878466,1,970840.0,420,0,0,0,0,0,0,...,4235090.0,659,4788674.0,3014688.0,478,0.0,1045000.0,2102260.0,2.98,2.98
1,2078768,0,1547887.0,1265,0,0,0,0,0,1,...,6394213.0,4248,9964534.0,4504160.0,730,0.0,1702932.0,2281412.0,1.98,1.98
2,66889,0,30301.0,0,0,0,0,0,0,0,...,545664.0,932,1853155.0,1020000.0,13,0.0,503000.0,0.0,0.99,0.99
3,3169202,1,3576488.0,4345,0,1,0,0,0,1,...,12919086.0,6110,14578830.0,6907604.0,6110,1565976.0,3787258.0,5026825.0,0.99,0.99
4,878159,0,41091.0,0,0,0,0,0,0,0,...,1431281.0,56,2462137.0,1000000.0,56,0.0,506000.0,0.0,0.99,0.99


In [3]:
test = pd.read_csv(path_test + 'tap_fun_test-fe-coor0.3.csv')
test.head()

,user_id,sr_infantry_atk_level,magic_add_value,cavalry_add_value,sr_rss_e_gather_level,sr_cavalry_atk_level,sr_march_size_level,sr_outpost_tier_4_level,sr_guest_troop_capacity_level,sr_shaman_atk_level,...,training_acceleration_reduce_value,wood_reduce_value,general_acceleration_add_value,wood_add_value,stone_add_value,general_acceleration_reduce_value,ivory_reduce_value,ivory_add_value,stone_reduce_value,pay_price
0,14933,0,0,0,0,0,0,0,0,0,...,0,138362.0,3,166415.0,0,0,0,0,0,0.0
1,14934,0,0,0,0,0,0,0,0,0,...,0,600.0,0,10000.0,0,0,0,0,0,0.0
2,14935,0,0,0,0,0,0,0,0,0,...,0,3700.0,0,10000.0,0,0,0,0,0,0.0
3,14936,0,60000,0,0,0,0,0,0,0,...,0,0.0,660,210000.0,200000,0,0,200000,0,0.0
4,14937,0,0,0,0,0,0,0,0,0,...,0,3700.0,0,11500.0,0,0,0,0,0,0.0


In [4]:
# 从原始数据中分离输入特征x和输出y
target = 'prediction_pay_price'
id = 'user_id'
train_y = train[target].values
train_X = train.drop([target, id], axis = 1)

# test_id = test[id]
test_X = test.drop([id], axis = 1)

## 数据标准化

In [5]:
# 分别初始化对特征和目标值的标准化器
ss_X = StandardScaler()
ss_y = StandardScaler()

# 分别对训练和测试数据的特征以及目标值进行标准化处理
train_X = ss_X.fit_transform(train_X)
test_X = ss_X.transform(test_X)

# 无论是否标准化 train_y 得到的预测结果相同
if Y_Standard:
    train_y = ss_y.fit_transform(train_y.reshape(-1, 1))

In [6]:
# 标准化数据还原 
def inverse_StandardScaler(predict):
    if Y_Standard:
        predict = ss_y.inverse_transform(predict)
        print('predict = ', predict)
    return predict

## 模型训练
- 直接调用 LightGBM 内嵌的交叉验证（cv），可对连续的 n_estimators 参数进行快速交叉验证
- 而 GridSearchCV 只能对有限个参数进行交叉验证

### 采用默认参数配置，看看模型是过拟合还是欠拟合

In [8]:
lgbm_1 = lgbm.LGBMRegressor(objective='regression',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)
lgbm_1.fit(train_X, train_y, eval_metric='l1')

D:\Applications\Anaconda\envs\python3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMRegressor(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.05,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_split_gain=0.0, n_estimators=20, n_jobs=-1, num_leaves=31,
       objective='regression', random_state=0, reg_alpha=0.0,
       reg_lambda=0.0, silent=True, subsample=1.0, subsample_for_bin=50000,
       subsample_freq=1)

In [14]:
# predict
# y_pred = lgbm_1.predict(test_X, num_iteration=lgbm_1.best_iteration_)
lgbm_1_pred = lgbm_1.predict(test_X)
lgbm_1_pred

array([-0.10146111, -0.10146111, -0.10146111, ..., -0.10146111,
       -0.10146111, -0.10146111])

In [25]:
lgbm_1_pred_train = lgbm_1.predict(train_X )
lgbm_1_pred_train

print('The rmse of prediction is:', mean_squared_error(train_y, lgbm_1_pred_train) ** 0.5)
print('Feature importances:', list(lgbm_1.feature_importances_))

The rmse of prediction is: 0.585378123593
Feature importances: [19, 18, 23, 3, 7, 0, 0, 0, 13, 0, 0, 6, 12, 1, 6, 0, 1, 0, 0, 0, 0, 0, 38, 13, 0, 0, 24, 0, 0, 2, 0, 14, 20, 0, 19, 38, 14, 18, 16, 26, 15, 16, 26, 24, 16, 16, 136]


In [15]:
# 标准化数据还原 
lgbm_1_pred = inverse_StandardScaler(lgbm_1_pred)

#输出预测后的数据
def generate_summit(predict):
    testPredict = test.copy()
    testPredict[target] = predict
    testPredict = testPredict[[id,target]]
    testPredict[target] = testPredict[target].apply(lambda x: x if x > 0 else 0)
    return testPredict

#输出预测后的数据
testPredict = generate_summit(lgbm_1_pred)
testPredict.head()

predict =  [ 33.85809542  33.85809542  33.85809542 ...,  33.85809542  33.85809542
  33.85809542]


,user_id,prediction_pay_price
0,14933,33.858095
1,14934,33.858095
2,14935,33.858095
3,14936,33.858095
4,14937,33.858095


In [16]:
#输出预测后的数据
testPredict.to_csv('%ssummit0.1-coor0.3-pay-lgbm-%s.csv' %(summit_path, out_id), index=False)
testPredict.shape

(828934, 2)

## 参数调优 
### 1. learning_rate / n_estimators

In [41]:
# 需要转换数据
train_y_1d = train_y.reshape(train_y.shape[0])
train_y_1d

array([-0.16597985, -0.16806931, -0.17013789, ..., -0.17013789,
       -0.01378312, -0.16393217])

In [39]:
%%time

estimator = lgbm.LGBMRegressor(objective='regression')
param_grid = {
    'learning_rate': [0.008, 0.01, 0.02],
    'n_estimators': [120, 130, 140, 145, 150, 155, 160]
}
 
lgbm_2 = GridSearchCV(estimator, param_grid)
lgbm_2.fit(train_X, train_y_1d)
 
print('Best parameters found by grid search are:', lgbm_2.best_params_)
# Best parameters found by grid search are: {'learning_rate': 0.01, 'n_estimators': 200}
# Best parameters found by grid search are: {'learning_rate': 0.01, 'n_estimators': 150}

Best parameters found by grid search are: {'learning_rate': 0.01, 'n_estimators': 150}
Wall time: 3min 33s


In [48]:
# 保存最佳参数
learning_rate = lgbm_2.best_params_['learning_rate']
n_estimators = lgbm_2.best_params_['n_estimators']
n_estimators

150

## 2：调整树的参数：num_leaves
(粗调，参数的步长为10；下一步是在粗调最佳参数周围，将步长降为2，进行精细调整)

In [53]:
%%time
estimator = lgbm.LGBMRegressor(objective='regression'
                              , learning_rate=learning_rate
                              , n_estimators=n_estimators)
param_grid = {
    'num_leaves': [26, 28, 30, 32, 34]
}
 
lgbm_3 = GridSearchCV(estimator, param_grid)
lgbm_3.fit(train_X, train_y_1d)

num_leaves = lgbm_3.best_params_['num_leaves']
print('Best parameters found by grid search are:', lgbm_3.best_params_)
# Best parameters found by grid search are: {'num_leaves': 30}

Best parameters found by grid search are: {'num_leaves': 32}
Wall time: 1min 14s


## 3：调整树的参数：max_depth & min_child_weight
(粗调，参数的步长为2；下一步是在粗调最佳参数周围，将步长降为1，进行精细调整)

In [59]:
%%time
estimator = lgbm.LGBMRegressor(objective='regression'
                              , learning_rate=learning_rate
                              , n_estimators=n_estimators
                              , num_leaves=num_leaves)
param_grid = {
    'max_depth': [5, 6, 7],
    'min_child_weight' : [1, 2, 3]
}
 
lgbm_4 = GridSearchCV(estimator, param_grid)
lgbm_4.fit(train_X, train_y_1d)

max_depth = lgbm_4.best_params_['max_depth']
min_child_weight = lgbm_4.best_params_['min_child_weight']
print('Best parameters are:', lgbm_4.best_params_)
# Best parameters are: {'max_depth': 5, 'min_child_weight': 1}

Best parameters found by grid search are: {'max_depth': 5, 'min_child_weight': 1}
Wall time: 55.7 s


## 4：调整树的参数：subsample 和 colsample_bytree
(粗调，参数的步长为0.1；下一步是在粗调最佳参数周围，将步长降为0.05，进行精细调整)

In [70]:
%%time
estimator = lgbm.LGBMRegressor(objective='regression',
                               learning_rate=learning_rate,
                               n_estimators=n_estimators,
                               num_leaves=num_leaves,
                               max_depth=max_depth,
                               min_child_weight=min_child_weight
                              )
param_grid = {
    'subsample': [0.85, 0.9, 0.95],
    'colsample_bytree' : [0.55, 0.6, 0.65]
}
lgbm_5 = GridSearchCV(estimator, param_grid)
lgbm_5.fit(train_X, train_y_1d)

subsample = lgbm_5.best_params_['subsample']
colsample_bytree = lgbm_5.best_params_['colsample_bytree']
print('Best parameters are:', lgbm_5.best_params_)
# Best parameters are: {'colsample_bytree': 0.6, 'subsample': 0.9}

Best parameters are: {'colsample_bytree': 0.55, 'subsample': 0.95}
Wall time: 36.3 s


## 5：调整正则化参数：reg_alpha 和 reg_lambda

In [75]:
%%time
estimator = lgbm.LGBMRegressor(objective='regression',
                               learning_rate=learning_rate,
                               n_estimators=n_estimators,
                               num_leaves=num_leaves,
                               max_depth=max_depth,
                               min_child_weight=min_child_weight,
                               subsample=subsample,
                               colsample_bytree=colsample_bytree
                              )
param_grid = {
    'reg_alpha': [0.038, 0.04, 0.042],
    'reg_lambda' : [0.17, 0.18, 0.19]
}
lgbm_5 = GridSearchCV(estimator, param_grid)
lgbm_5.fit(train_X, train_y_1d)

reg_alpha = lgbm_5.best_params_['reg_alpha']
reg_lambda = lgbm_5.best_params_['reg_lambda']
print('Best parameters are:', lgbm_5.best_params_)
# Best parameters are: {'reg_alpha': 0.05, 'reg_lambda': 0.2}

Best parameters are: {'reg_alpha': 0.042, 'reg_lambda': 0.18}
Wall time: 31.8 s


## 6. 再次调优 learning_rate / n_estimators

In [80]:
%%time
estimator = lgbm.LGBMRegressor(objective='regression',
                               num_leaves=num_leaves,
                               max_depth=max_depth,
                               min_child_weight=min_child_weight,
                               subsample=subsample,
                               colsample_bytree=colsample_bytree,
                               reg_alpha=reg_alpha,
                               reg_lambda=reg_lambda
                              )
param_grid = {
    'learning_rate': [0.01],
    'n_estimators' : [173, 175, 176,177]
}
lgbm_5 = GridSearchCV(estimator, param_grid)
lgbm_5.fit(train_X, train_y_1d)

learning_rate = lgbm_5.best_params_['learning_rate']
n_estimators = lgbm_5.best_params_['n_estimators']
print('Best parameters are:', lgbm_5.best_params_)
# Best parameters are: {'learning_rate': 0.01, 'n_estimators': 180}
# Wall time: 36.1 s

Best parameters are: {'learning_rate': 0.01, 'n_estimators': 176}
Wall time: 18.2 s


## 7 用最佳模型测试

In [82]:
lgbm_5_pred_train = lgbm_5.predict(train_X )
print('The rmse of prediction is:', mean_squared_error(train_y, lgbm_5_pred_train) ** 0.5)

The rmse of prediction is: 0.51561109904


In [83]:
lgbm_5_pred = lgbm_5.predict(test_X)
# 标准化数据还原 
lgbm_5_pred = inverse_StandardScaler(lgbm_5_pred)
#输出预测后的数据
testPredict = generate_summit(lgbm_5_pred)
testPredict.head()

predict =  [ 18.22507667  18.22507667  18.22507667 ...,  18.22507667  18.22507667
  18.22507667]


,user_id,prediction_pay_price
0,14933,18.225077
1,14934,18.225077
2,14935,18.225077
3,14936,18.225077
4,14937,18.225077


In [86]:
#输出预测后的数据
out_id += 1
testPredict.to_csv('%ssummit0.1-coor0.3-pay-lgbm-%s.csv' %(summit_path, out_id), index=False)
testPredict.shape

(828934, 2)